In [1]:
import os, sys
import time
import datetime
from typing import List, Optional, Tuple
from logging import ERROR, INFO
import flwr as fl
from flwr.common.logger import log
from flwr.common import EvaluateIns, Parameters, parameters_to_ndarrays, ndarrays_to_parameters
from flwr.server.strategy import FedAvg
from flwr.simulation import start_simulation
from flwr.server.criterion import Criterion
from flwr.server.client_proxy import ClientProxy
from flwr.server.client_manager import ClientManager, SimpleClientManager
from typing import Callable, Dict, List, Optional, Tuple, Union
from flwr.common import EvaluateRes, FitRes, MetricsAggregationFn, NDArrays, Parameters, Scalar
from flwr.server.client_proxy import ClientProxy
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import load_model
import util_flwr
import const_flwr
import random
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg
import communication_costs

In [2]:
NUM_ROUNDS_new_values = [1] #5, 10, 50
LEARNING_RATE_new_values = [0.001,0.01,0.1] # 0.1, 0.01, 0.001
BATCH_SIZE_new_values = [256] #4, 8, 16, 32, 64, 128, 256
LOCAL_EPOCHS_new_values = [1,2,5,10,50] #2, 5, 10, 50

TOTAL_CLIENTS = const_flwr.NUM_CLIENTS
#util_flwr.set_num_rounds(NUM_ROUNDS_new_values)
#EPSILON = 1
#PARAMETRO_RESTRICAO = 0.3

In [3]:


for rd in NUM_ROUNDS_new_values:
  for lr in LEARNING_RATE_new_values:
    for bz in BATCH_SIZE_new_values:
      for le in LOCAL_EPOCHS_new_values:
        
        # Creating files for the results
        NO_SELECTION_AGGREGATED_EVALUATOIN_DIR = const_flwr.DIR_EVAL_COSTS_PER_ROUND_NO_SELECTION + "/"+ str(rd) + "_" + str(lr) + "_" + str(bz) + "_" + str(le) + "/"
        NO_SELECTION_AGGREGATED_FIT_DIR = const_flwr.DIR_FIT_COSTS_PER_ROUND_NO_SELECTION + "/"+ str(rd) + "_" + str(lr) + "_" + str(bz) + "_" + str(le) + "/"

        

        if not os.path.exists(const_flwr.DIR_MODELS):
            os.makedirs(const_flwr.DIR_MODELS)

        if not os.path.exists(NO_SELECTION_AGGREGATED_EVALUATOIN_DIR):
            os.makedirs(NO_SELECTION_AGGREGATED_EVALUATOIN_DIR)

        if not os.path.exists(NO_SELECTION_AGGREGATED_FIT_DIR):
            os.makedirs(NO_SELECTION_AGGREGATED_FIT_DIR)


        # O QUE FAZ ESSA VERSÃO?
        # implementa os custos de comunicação sem a seleção de clientes


        def main() -> None:
            
            #Initiating and compiling the global model
            
            GLOBAL_MODEL = tf.keras.Sequential([
                    tf.keras.layers.Dense(const_flwr.NEURONS_FIRST_LAYER, activation=const_flwr.ACTIVE_FN_FIRST_LAYER, input_shape=(const_flwr.NUM_FEATURES, )),
                    tf.keras.layers.Dense(const_flwr.NEURONS_LAST_LAYER, activation=const_flwr.ACTIVE_FN_LAST_LAYER)
                ])
                
            GLOBAL_MODEL.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr) , #tf.keras.optimizers.Adagrad, #tf.keras.optimizers.Adam(learning_rate=const_flwr.LEARING_RATE), #,
                        loss=const_flwr.LOSS_v2,
                        metrics=const_flwr.METRICS_v2
                        )
            
            # Defining the centralized dataset used to pre-train the global model
            x_train, y_train, x_test, y_test= util_flwr.load_TMD_Centralized_dataset_OneHot() 
                
            # Pre-training the global model
            GLOBAL_MODEL.fit(x_train, y_train, batch_size=bz, epochs=le)
            
            # Saving the global model 
            initial_global_model_filename = const_flwr.DIR_MODELS + "/"+ str(rd) + "_" + str(lr) + "_" + str(bz) + "_" + str(le) + "/" + './initial_global_model.h5'
            GLOBAL_MODEL.save(initial_global_model_filename)
            util_flwr.set_model_size(sys.getsizeof(GLOBAL_MODEL))

            def client_fn(cid: str) -> fl.client.Client:
                """Function to create clients for the simulation"""

                # Loading training and testing datasets for each client
                x_train, y_train, x_test, y_test = util_flwr.load_TMDataset_OneHot(cid)    
                
                initial_global_model_filename = const_flwr.DIR_MODELS + "/"+ str(rd) + "_" + str(lr) + "_" + str(bz) + "_" + str(le) + "/" + './initial_global_model.h5'
                local_model_filename = const_flwr.DIR_MODELS + "/"+ str(rd) + "_" + str(lr) + "_" + str(bz) + "_" + str(le) + "/" + f'./model_client_{cid}.h5'
                   
                # Trying to find if there is a local model, and if there is not, a copy of the global model is created and saved as client local model
                try:
                    if os.path.exists(local_model_filename):
                        loaded_model = load_model(local_model_filename)
                        print(f"CLIENT {cid} LOADED")
                    else:
                        loaded_model = load_model(initial_global_model_filename)
                        loaded_model.save(local_model_filename)
                        print(f"CLIENT {cid} CREATED")
                        
                except Exception as e:
                    print(f"Erro ao carregar o modelo: {e}")
                
                client_id = cid

                return FlwrClient(client_id, loaded_model, x_train, y_train, x_test, y_test)

        
            def get_evaluate_fn():
                
                def evaluate(server_round, parameters, config): 
                    """For each round, evaluate global model parameters using an evaluation function."""
                    
                    initial_global_model_filename = const_flwr.DIR_MODELS + "/"+ str(rd) + "_" + str(lr) + "_" + str(bz) + "_" + str(le) + "/" + './initial_global_model.h5'
                    aggregated_global_model_filename = const_flwr.DIR_MODELS + "/"+ str(rd) + "_" + str(lr) + "_" + str(bz) + "_" + str(le) + "/" + './aggregated_global_model.h5'
                    
                    try:
                        if os.path.exists(aggregated_global_model_filename):
                            model = load_model(aggregated_global_model_filename)
                        else:
                            model = load_model(initial_global_model_filename)
                            model.save(aggregated_global_model_filename)
                            
                    except Exception as e:
                        print(f"Erro ao carregar o modelo agregado: {e}")

                    model.set_weights(parameters)
                    
                    #Geting the performance metrics for each round
                    loss_value, accuracy_value, precision_value, recall_value = model.evaluate(x_test, y_test, batch_size=bz) #, callbacks=tensorboard_callback_server_evaluating)     
                    
                    model.save(aggregated_global_model_filename)

                    return loss_value, {"categorical_accuracy": accuracy_value, "precision": precision_value, "recall": recall_value}

                return evaluate

            def fit_config(server_round: int):
                """ Return training configuration dict for each round """
                config = {
                    "batch_size": bz,
                    "local_epochs": le   
                }
                return config

            def evaluate_config(server_round: int):
                """ Return evaluation configuration dict for each round """
                return {"server_round": server_round}

            

            fl.simulation.start_simulation(
                        client_fn=client_fn,
                        num_clients=const_flwr.NUM_CLIENTS,
                        #client_manager=FedTMD_ClientManager(),
                        config=fl.server.ServerConfig(num_rounds=rd),
                        strategy=FedTMDStrategy(
                            fraction_fit=1,
                            fraction_evaluate=1,
                            evaluate_fn=get_evaluate_fn(),
                            on_fit_config_fn=fit_config,
                            on_evaluate_config_fn=evaluate_config,
                            initial_parameters=ndarrays_to_parameters(GLOBAL_MODEL.get_weights())
                        ),
                        
                    )

       


        class FlwrClient(fl.client.NumPyClient):
            def __init__(self, client_id, model, x_train, y_train, x_test, y_test) -> None:
                super().__init__()
                self.client_id = client_id
                self.model = model
                self.x_train, self.y_train = x_train, y_train
                self.x_test, self.y_test = x_test, y_test

                # self.clients_fit_log_dir = os.path.join(const_flwr.TENSORBOARD_SERVER_BASE_LOG_DIR, f"clients_fit_logs/client_{client_id}")
                # self.clients_evaluate_log_dir = os.path.join(const_flwr.TENSORBOARD_SERVER_BASE_LOG_DIR, f"clients_evaluate_logs/client_{client_id}")

            def get_properties(self, config):
                
                # Supondo que y_train é o seu ndarray de rótulos no formato one-hot
                tm = np.argmax(self.y_train, axis=1)
                num_unique_tm = np.unique(tm).size
                # local_size = len(self.x_train)
                #contribution = (num_unique_tm * train_size) / (num_unique_tm + train_size)
                # print(f"CLIENT {self.client_id} | UNIQUE TMD LABELS: {num_unique_tm} | CONTRIBUTION: {contribution}")
                res = {"num_unique_tm": num_unique_tm}
                return res

            def get_parameters(self, config):
                return self.model.get_weights()

            def fit(self, parameters, config):
                # tensorboard_callback_client_training = TensorBoard(log_dir=self.clients_fit_log_dir)
                # wt = parameters
                # wk = self.model.get_weights()


                self.model.set_weights(parameters)  

                # self.model.fit(self.x_train, self.y_train, epochs=const_flwr.LOCAL_EPOCHS, batch_size=const_flwr.BATCH_SIZE, callbacks=[tensorboard_callback_client_training])                                    
                
                # Get hyperparameters for this round
                batch_size: int = config["batch_size"]
                epochs: int = config["local_epochs"] # VERSÃO ORIGINAL
                # epsilon = 0.5
                # epochs = epsilon / 2 * np.linalg.norm(wk - wt) ** 2

                # Train the model using hyperparameters from config
                history = self.model.fit(
                    self.x_train,
                    self.y_train,
                    batch_size,
                    epochs,
                    # validation_split=0.1,
                    # callbacks=[tensorboard_callback_client_training]
                )
                
                results = {
                    "loss": history.history["loss"][0],
                    "categorical_accuracy": history.history["categorical_accuracy"][0],
                    "precision": history.history["precision"][0],
                    "recall": history.history["recall"][0],
                }       
                
                updated_parameters = self.model.get_weights()
                
                return updated_parameters, len(self.x_train), results

            def evaluate(self, parameters, config):
                self.model.set_weights(parameters)
                server_round: int = config["server_round"]
                # print(f"CLIENT {self.client_id} | EVALUATING PARAMETERS | ROUND {server_round}")
                loss_value, accuracy_value, precision_value, recall_value = self.model.evaluate(self.x_test, self.y_test, batch_size=bz)
                
                return loss_value, len(self.x_test), {"categorical_accuracy": accuracy_value, "precision": precision_value, "recall": recall_value}


        class FedTMD_ClientManager(SimpleClientManager):
            print("USING FedTMD_ClientManager")
            def __init__(self) -> None:
                super().__init__()
            
            def sample(self, num_clients, min_num_clients, criterion: Criterion) -> List[ClientProxy]:
                """Sample a number of Flower ClientProxy instances."""
                #print("USING SAMPLING FROM FedTMD_ClientManager")

                # Block until at least num_clients are connected.
                if min_num_clients is None:
                    min_num_clients = num_clients
                self.wait_for(min_num_clients)

                # Sample clients which meet the criterion
                available_cids = list(self.clients)

                if criterion is not None:
                    available_cids = [
                        cid for cid in available_cids if criterion.select(self.clients[cid])
                    ]


                if num_clients > len(available_cids):
                    log(
                        INFO,
                        "Sampling failed: number of available clients"
                        " (%s) is less than number of requested clients (%s).",
                        len(available_cids),
                        num_clients,
                    )
                    return []

                # Making sure the seed of random is really random
                now = datetime.datetime.now()
                seed = now.microsecond
                random.seed(seed)
                sampled_cids = random.sample(available_cids, num_clients) 
                # print("SAMPLED CIDS: ")
                # print(sampled_cids)
                util_flwr.set_num_clients_per_round(num_clients)
                
                          
                return [self.clients[cid] for cid in sampled_cids]



        class FedTMDStrategy(FedAvg):           
            
            def configure_evaluate(self, server_round: int, parameters: Parameters, client_manager: SimpleClientManager) -> List[Tuple[ClientProxy, EvaluateIns]]:
                """Configure the next round of evaluation."""
                # Do not configure federated evaluation if fraction eval is 0.
                if self.fraction_evaluate == 0.0:
                    return []

                # Parameters and config
                config = {}
                if self.on_evaluate_config_fn is not None:
                    # Custom evaluation config function provided
                    config = self.on_evaluate_config_fn(server_round)
                evaluate_ins = EvaluateIns(parameters, config)

                # Sample clients
                sample_size, min_num_clients = self.num_evaluation_clients(
                    client_manager.num_available()
                )
                clients = client_manager.sample(
                    num_clients=sample_size, 
                    min_num_clients=min_num_clients
                )

                # Return client/config pairs
                return [(client, evaluate_ins) for client in clients]

            def configure_fit(self, server_round, parameters, client_manager: SimpleClientManager):
                """Configure the next round of training."""
                #print("USING FedTMDStrategy")
                config = {}
                if self.on_fit_config_fn is not None:
                    # Custom fit config function provided
                    config = self.on_fit_config_fn(server_round)
                fit_ins = fl.common.typing.FitIns(parameters, config)

                # STORING PARAMETER SIZE 
                util_flwr.set_parameters_size(len(parameters.tensors))

                # Sample clients
                sample_size, min_num_clients = self.num_fit_clients(
                    client_manager.num_available()
                )
                clients = client_manager.sample(
                    num_clients=sample_size,
                    min_num_clients=min_num_clients
                )

                # Return client/config pairs
                return [(client, fit_ins) for client in clients]

            def aggregate_fit(self, server_round, results, failures):

                print(f"AGGREGATING FIT METRICS AND PARAMETERS | ROUND {server_round}")

        
                weights_results = [
                    (parameters_to_ndarrays(r.parameters), r.num_examples)
                    for _, r in results
                ]

                aggregated_parameters = ndarrays_to_parameters(aggregate(weights_results))

                # Weigh accuracy of each client by number of examples used       
                accuracies = [r.metrics["categorical_accuracy"] * r.num_examples for _, r in results]
                accuracy_examples = [r.num_examples for _, r in results]
                aggregated_accuracy = sum(accuracies) / sum(accuracy_examples)

                precision = [r.metrics["precision"] * r.num_examples for _, r in results]
                precision_examples = [r.num_examples for _, r in results]
                aggregated_precision = sum(precision) / sum(precision_examples)

                recall = [r.metrics["recall"] * r.num_examples for _, r in results]
                recall_examples = [r.num_examples for _, r in results]
                aggregated_recall = sum(recall) / sum(recall_examples)
                
              
                #NUM_CLIENTS_PER_ROUND = const_flwr.NUM_CLIENTS
                #PARAMETERS_SIZE = util_flwr.get_parameters_size()
                #ROUND_COMMUNICATION_COSTS = (NUM_CLIENTS_PER_ROUND * PARAMETERS_SIZE) #+ (TOTAL_CLIENTS * PARAMETERS_SIZE)
                # TOTAL_COMMUNICATION_COSTS = (NUM_CLIENTS_PER_ROUND * PARAMETERS_SIZE) + (TOTAL_CLIENTS * PARAMETERS_SIZE)
                
                COST = communication_costs.calculate_communication_costs()
                BOUND = communication_costs.calculate_upper_bound()
                
                try:
                    df_resultados = pd.read_csv(NO_SELECTION_AGGREGATED_FIT_DIR + const_flwr.FILE_AGGREGATED_FITTING_METRICS)
                except FileNotFoundError:
                    df_resultados = pd.DataFrame(columns=['round', 'accuracy', 'precision', 'recall', 'cost', 'bound'])


                df_feature = pd.DataFrame(
                    {'round': [server_round], 'accuracy': [aggregated_accuracy], 'precision': [aggregated_precision], 'recall': [aggregated_recall], 'cost': [COST], 'bound': [BOUND]})
                
                # Concatene o novo dataframe com o dataframe existente
                df_resultados = pd.concat([df_resultados, df_feature], ignore_index=True)

                df_resultados.to_csv(NO_SELECTION_AGGREGATED_FIT_DIR + const_flwr.FILE_AGGREGATED_FITTING_METRICS, index=False)
                
 
                return aggregated_parameters, {"categorical_accuracy": aggregated_accuracy, "precision": aggregated_precision, "recall":aggregated_recall}
                
            def aggregate_evaluate(self, server_round, results, failures) :
                """Aggregate evaluation accuracy using weighted average."""
                # print(f"AGGREGATING EVALUATION METRICS | ROUND {server_round}")

                aggregated_loss = weighted_loss_avg(
                    [
                        (evaluate_res.num_examples, evaluate_res.loss)
                        for _, evaluate_res in results
                    ]
                )
                
                # Weigh accuracy of each client by number of examples used       
                accuracies = [r.metrics["categorical_accuracy"] * r.num_examples for _, r in results]
                accuracy_examples = [r.num_examples for _, r in results]
                aggregated_accuracy = sum(accuracies) / sum(accuracy_examples)

                precision = [r.metrics["precision"] * r.num_examples for _, r in results]
                precision_examples = [r.num_examples for _, r in results]
                aggregated_precision = sum(precision) / sum(precision_examples)

                recall = [r.metrics["recall"] * r.num_examples for _, r in results]
                recall_examples = [r.num_examples for _, r in results]
                aggregated_recall = sum(recall) / sum(recall_examples)

                
                try:
                    df_resultados = pd.read_csv(NO_SELECTION_AGGREGATED_EVALUATOIN_DIR + const_flwr.FILE_AGGREGATED_EVALUATION_METRICS)
                except FileNotFoundError:
                    df_resultados = pd.DataFrame(columns=['round', 'loss', 'accuracy', 'precision', 'recall'])


                df_feature = pd.DataFrame(
                    {'round': [server_round], 'loss': [aggregated_loss], 'accuracy': [aggregated_accuracy], 'precision': [aggregated_precision], 'recall': [aggregated_recall]})
                
                # Concatene o novo dataframe com o dataframe existente
                df_resultados = pd.concat([df_resultados, df_feature], ignore_index=True)

                df_resultados.to_csv(NO_SELECTION_AGGREGATED_EVALUATOIN_DIR + const_flwr.FILE_AGGREGATED_EVALUATION_METRICS, index=False)
                

                return aggregated_loss, {"categorical_accuracy": aggregated_accuracy, "precision": aggregated_precision, "recall":aggregated_recall}


        

        if __name__ == "__main__":
            main()



          

USING FedTMD_ClientManager
13/13 [==============================] - 2s 7ms/step - loss: 1.6338 - categorical_accuracy: 0.2149 - precision: 0.0227 - recall: 3.0488e-04


2024-02-26 11:15:42,735	INFO worker.py:1621 -- Started a local Ray instance.


13/13 [==============================] - 1s 7ms/step - loss: 1.6046 - categorical_accuracy: 0.2435 - precision: 0.1000 - recall: 9.1436e-04
(launch_and_fit pid=5392) CLIENT 3 CREATED
(launch_and_fit pid=5392) 
(launch_and_fit pid=5392) 1/2 [==============>...............] - ETA: 1s - loss: 1.6886 - categorical_accuracy: 0.2305 - precision: 0.0000e+00 - recall: 0.0000e+00
(launch_and_fit pid=5392) 2/2 [==============================] - 2s 25ms/step - loss: 1.6907 - categorical_accuracy: 0.2250 - precision: 0.0000e+00 - recall: 0.0000e+00
(launch_and_fit pid=7348) 1/1 [==============================] - ETA: 0s - loss: 1.7504 - categorical_accuracy: 0.0366 - precision: 0.0000e+00 - recall: 0.0000e+00
(launch_and_fit pid=7348) 1/3 [=========>....................] - ETA: 3s - loss: 1.6465 - categorical_accuracy: 0.1719 - precision: 0.0000e+00 - recall: 0.0000e+00
(launch_and_fit pid=5392) 1/2 [==============>...............] - ETA: 1s - loss: 1.6624 - categorical_accuracy: 0.3320 - precisio

2024-02-26 11:16:49,888	INFO worker.py:1621 -- Started a local Ray instance.


13/13 [==============================] - 1s 6ms/step - loss: 1.5514 - categorical_accuracy: 0.2710 - precision: 0.6667 - recall: 0.0091
(launch_and_fit pid=8496) CLIENT 13 CREATED
(launch_and_fit pid=8496) Epoch 1/2
(launch_and_fit pid=8496) 
(launch_and_fit pid=8496) 1/1 [==============================] - ETA: 0s - loss: 1.6770 - categorical_accuracy: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
(launch_and_fit pid=8496) 1/1 [==============================] - 2s 2s/step - loss: 1.6770 - categorical_accuracy: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
(launch_and_fit pid=8496) 
(launch_and_fit pid=8496) 1/1 [==============================] - ETA: 0s - loss: 1.6686 - categorical_accuracy: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
(launch_and_fit pid=8256) 1/4 [======>.......................] - ETA: 5s - loss: 1.5230 - categorical_accuracy: 0.2812 - precision: 0.0000e+00 - recall: 0.0000e+00
(launch_and_fit pid=8496) 1/1 [============================

...

13/13 [==============================] - 1s 6ms/step - loss: 0.8762 - categorical_accuracy: 0.6772 - precision: 0.9006 - recall: 0.4282
(launch_and_fit pid=4620) CLIENT 7 CREATED
(launch_and_fit pid=10164) Epoch 1/5
(launch_and_fit pid=10164) 
(launch_and_fit pid=10164) 1/1 [==============================] - ETA: 0s - loss: 2.5111 - categorical_accuracy: 0.2857 - precision: 0.0800 - recall: 0.0286
(launch_and_fit pid=10164) 1/1 [==============================] - 2s 2s/step - loss: 2.5111 - categorical_accuracy: 0.2857 - precision: 0.0800 - recall: 0.0286
(launch_and_fit pid=10164) 
(launch_and_fit pid=10164) 1/1 [==============================] - ETA: 0s - loss: 1.0955 - categorical_accuracy: 0.6857 - precision: 0.5172 - recall: 0.2143
(launch_and_fit pid=10164) 1/1 [==============================] - 0s 21ms/step - loss: 1.0955 - categorical_accuracy: 0.6857 - precision: 0.5172 - recall: 0.2143
(launch_and_fit pid=10164) 
(launch_and_fit pid=10164) 1/1 [==============================] 

2024-02-26 11:30:22,032	INFO worker.py:1621 -- Started a local Ray instance.


13/13 [==============================] - 1s 7ms/step - loss: 0.7746 - categorical_accuracy: 0.7105 - precision: 0.8893 - recall: 0.5483
(launch_and_fit pid=8092) CLIENT 5 CREATED
(launch_and_fit pid=8092) Epoch 1/10
(launch_and_fit pid=10452) 
(launch_and_fit pid=10452) 1/2 [==============>...............] - ETA: 2s - loss: 3.8685 - categorical_accuracy: 0.0781 - precision: 0.0249 - recall: 0.0195
(launch_and_fit pid=10452) 2/2 [==============================] - 2s 27ms/step - loss: 3.0137 - categorical_accuracy: 0.1759 - precision: 0.0929 - recall: 0.0602
(launch_and_fit pid=10452) 
(launch_and_fit pid=10452) 1/2 [==============>...............] - ETA: 0s - loss: 0.9749 - categorical_accuracy: 0.6758 - precision: 0.7119 - recall: 0.4922
(launch_and_fit pid=1460) 1/3 [=========>....................] - ETA: 4s - loss: 2.3400 - categorical_accuracy: 0.3359 - precision: 0.3067 - recall: 0.1953
(launch_and_fit pid=1460) 2/3 [===================>..........] - ETA: 0s - loss: 0.2118 - catego

2024-02-26 11:31:34,400	INFO worker.py:1621 -- Started a local Ray instance.


13/13 [==============================] - 1s 7ms/step - loss: 0.5650 - categorical_accuracy: 0.7927 - precision: 0.8592 - recall: 0.7178
(launch_and_fit pid=12732) CLIENT 2 CREATED
(launch_and_fit pid=11492) Epoch 1/50
(launch_and_fit pid=12732) 
(launch_and_fit pid=12732) 1/3 [=========>....................] - ETA: 3s - loss: 2.1472 - categorical_accuracy: 0.4102 - precision: 0.3900 - recall: 0.3047
(launch_and_fit pid=11492) 3/3 [==============================] - 2s 17ms/step - loss: 0.7114 - categorical_accuracy: 0.7210 - precision: 0.8484 - recall: 0.6186
(launch_and_fit pid=11492) 3/3 [==============================] - 0s 14ms/step - loss: 0.5627 - categorical_accuracy: 0.7615 - precision: 0.9070 - recall: 0.6833
(launch_and_fit pid=2636) 1/1 [==============================] - ETA: 0s - loss: 1.8665 - categorical_accuracy: 0.6449 - precision: 0.6731 - recall: 0.4907
(launch_and_fit pid=2636) 1/1 [==============================] - ETA: 0s - loss: 0.9546 - categorical_accuracy: 0.775